In [1]:
import pandas as pd
import os
import akvut

query = """
query MyQuery($id: ID!) {
  course(id: $id) {
    enrollmentsConnection {
      nodes {
        user {
          sisId
        }
        type
      }
    }
    sisId
  }
}
"""

# Lage liste for å deaktivere (eller reaktivere) alle i eit emne
Hent alle i emnet i dag, og lag ei CSV-fil som vi kan importere med SIS-IMPORT (etter at eg haar rydda i den).

In [2]:
emne = {'id': '22411'}
resultat = akvut.query_canvas_graphql(query, emne)

Så lager eg to lister:

- **aktiver.csv** som aktiverer alle studentane i eit emne

- **deaktiver.csv** som deaktiverer alle studentane i eit emne

In [4]:
data = resultat['data']['course']['enrollmentsConnection']['nodes']
liste1 = []
liste2 = []
liste3 = []
for d in data:
    if d['type']=="StudentEnrollment":
        d['type'] = "student"
    if d['type']=="TeacherEnrollment":
        d['type'] = "teacher"
    if d['type']=="StudentViewEnrollment":
        d['type'] = "observer"
    liste1.append([d['user']['sisId'], d['type']])
    liste2.append([d['user']['sisId'], d['type'], resultat['data']['course']['sisId'], 'completed'])
    liste3.append([d['user']['sisId'], d['type'], resultat['data']['course']['sisId'], 'active'])
personar = pd.DataFrame(liste1, columns=['sisId', 'type'])
deaktiver = pd.DataFrame(liste2, columns=['sisId', 'type', 'course', 'workflow_state'])
deaktiver.rename(columns={'sisId': 'user_id', 'type': 'role', 'course': 'course_id', 'workflow_state': 'status'}, inplace=True)
deaktiver.to_csv("deaktiver.csv", index=False)
aktiver = pd.DataFrame(liste3, columns=['sisId', 'type', 'course', 'workflow_state'])
aktiver.rename(columns={'sisId': 'user_id', 'type': 'role', 'course': 'course_id', 'workflow_state': 'status'}, inplace=True)
aktiver.to_csv("aktiver.csv", index=False)